## Imports

In [1]:
import sys
sys.setrecursionlimit(10**4)

from functools import cache

In [2]:
day = "17"

In [3]:
with open(f"input_j{day}.txt") as f:
# with open("test.txt") as f:
    grid = f.readlines()
    grid = list(map(lambda x: x.replace("\n", ""), grid))
    grid = list(map(lambda x: [int(case) for case in x], grid))

n, m = len(grid), len(grid[0])
print(n, m)

141 141


In [4]:
nb_same_dir_max = 3
directions = {'r': 0, 'l': 1, 'u': 2, 'd': 3}

## Part 1

In [5]:
distances_optimales = [
    [
        [
            [
                float("inf") for _ in range(nb_same_dir_max+1)
            ] for _ in directions.keys()
        ] for _ in range(m)
    ] for _ in range(n)
]

for dir in directions.keys():
    for nb_same_dir in range(1, nb_same_dir_max):
        distances_optimales[n-1][m-1][directions[dir]][nb_same_dir] = grid[n-1][m-1]

### Basic functions

In [6]:
def possible_directions(dir, nb_same_dir):
    directions = []
    if nb_same_dir < nb_same_dir_max:
        directions.append((dir, nb_same_dir+1))
    if dir == 'r':
        directions.extend([('u', 1), ('d', 1)])
    if dir == 'l':
        directions.extend([('u', 1), ('d', 1)])
    if dir == 'u':
        directions.extend([('l', 1), ('r', 1)])
    if dir == 'd':
        directions.extend([('l', 1), ('r', 1)])
    return directions

In [7]:
def next_case(x, y, dir, nb_same_dir):
    if dir == 'd' and x < n-1:
        return x+1, y, dir, nb_same_dir
    if dir == 'u' and x > 0:
        return x-1, y, dir, nb_same_dir
    if dir == 'r' and y < m-1:
        return x, y+1, dir, nb_same_dir
    if dir == 'l' and y > 0:
        return x, y-1, dir, nb_same_dir
    return False

### Essai infructueux

In [8]:
# Trop long

# @cache
def chemin_optimal_from(x, y, dir, nb_same_dir, deja_vus):

    if distances_optimales[x][y][directions[dir]][nb_same_dir] != float("inf"):
        return distances_optimales[x][y][directions[dir]][nb_same_dir]

    new_deja_vus = deja_vus.copy()
    new_deja_vus.add((x, y, dir, nb_same_dir))

    possible_dirs = possible_directions(dir, nb_same_dir)
    next_cases = [
        next_case(x, y, new_dir[0], new_dir[1]) for new_dir in possible_dirs if next_case(x, y, new_dir[0], new_dir[1])
    ]
    next_cases = [case for case in next_cases if case not in new_deja_vus]
    if (next_cases == []):
    # if (next_cases == []) or (len(new_deja_vus) > 5*n):
        return float('inf')

    distance_optimale = grid[x][y] + min(
        chemin_optimal_from(
            new_x, new_y, new_dir, new_nb_same_dir, new_deja_vus
        ) for new_x, new_y, new_dir, new_nb_same_dir in next_cases
    )
    distances_optimales[x][y][directions[dir]][nb_same_dir] = distance_optimale

    return distance_optimale

In [9]:
# a = chemin_optimal_from(0, 0, 'r', 1, set())
# b = chemin_optimal_from(0, 0, 'd', 1, set())
# min(a, b) - grid[0][0]

### Dijsktra

On passe à Dijkstra, recodé à la main J10

In [10]:
def find_min(Q):
    mini = float('inf')
    for node in Q:
        if distances_optimales[node[0]][node[1]][directions[node[2]]][node[3]] <= mini:
            mini = distances_optimales[node[0]][node[1]][directions[node[2]]][node[3]]
            sommet = node
    return sommet

In [11]:
def update_distances(node_1, node_2):
    if (
        distances_optimales[node_2[0]][node_2[1]][directions[node_2[2]]][node_2[3]]
    ) > (
        distances_optimales[node_1[0]][node_1[1]][directions[node_1[2]]][node_1[3]] + grid[node_2[0]][node_2[1]]
    ):
        distances_optimales[node_2[0]][node_2[1]][directions[node_2[2]]][node_2[3]] = (
            distances_optimales[node_1[0]][node_1[1]][directions[node_1[2]]][node_1[3]] + grid[node_2[0]][node_2[1]]
        )

In [12]:
def dijkstra(function_possible_dirs, part=1):
    # Q = [
    #     (x, y, dir, nb_same_dir+1) for x in range(n) for y in range(n) for dir in directions.keys() for nb_same_dir in range(nb_same_dir_max)
    # ]
    if part == 1:
        Q = set([(n-1, m-1, dir, 1) for dir in directions.keys()])
    if part == 2:
        Q = set([(n-1, m-1, dir, 0) for dir in directions.keys()])
    not_Q = set()
    compteur = 0

    while len(Q) > 0:
        compteur += 1
        if compteur % 20000 == 0:
            print(compteur // 20000)
        node_1 = find_min(Q)
        Q.remove(node_1)
        not_Q.add(node_1)
        possible_dirs = function_possible_dirs(node_1[2], node_1[3])
        voisins = [
            next_case(node_1[0], node_1[1], new_dir[0], new_dir[1]) for new_dir in possible_dirs if next_case(node_1[0], node_1[1], new_dir[0], new_dir[1])
    ]
        for node_2 in voisins:
            update_distances(node_1, node_2)
            if node_2 not in not_Q:
                Q.add(node_2)



### Application

In [13]:
dijkstra(possible_directions)

1
2
3
4
5
6
7
8
9
10
11


In [14]:
a = min(distances_optimales[0][0][directions['l']][:])
b = min(distances_optimales[0][0][directions['u']][:])
min(a, b) - grid[0][0]

916

## Part 2

### Changements

In [15]:
nb_same_dir_min = 4
nb_same_dir_max = 10

In [16]:
def new_possible_directions(dir, nb_same_dir):
    directions = []
    if nb_same_dir < nb_same_dir_max:
        directions.append((dir, nb_same_dir+1))
    if nb_same_dir >= nb_same_dir_min and dir == 'r':
        directions.extend([('u', 1), ('d', 1)])
    if nb_same_dir >= nb_same_dir_min and dir == 'l':
        directions.extend([('u', 1), ('d', 1)])
    if nb_same_dir >= nb_same_dir_min and dir == 'u':
        directions.extend([('l', 1), ('r', 1)])
    if nb_same_dir >= nb_same_dir_min and dir == 'd':
        directions.extend([('l', 1), ('r', 1)])
    return directions

### Application

In [17]:
distances_optimales = [
    [
        [
            [
                float("inf") for _ in range(nb_same_dir_max+1)
            ] for _ in directions.keys()
        ] for _ in range(m)
    ] for _ in range(n)
]

for dir in directions.keys():
    distances_optimales[n-1][m-1][directions[dir]][0] = grid[n-1][m-1]

In [18]:
dijkstra(new_possible_directions, part=2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [19]:
a = min(distances_optimales[0][0][directions['l']][nb_same_dir_min:])
b = min(distances_optimales[0][0][directions['u']][nb_same_dir_min:])
min(a, b) - grid[0][0]

1067